In [67]:
import tensorflow as tf
import os

In [68]:
data_dir = '../../../../Other/datasets/cats_vs_dogs'
train_cats_dir = data_dir + '/train/cats/'
train_dogs_dir = data_dir + '/train/dogs/'
test_cats_dir = data_dir + '/valid/cats/'
test_dogs_dir = data_dir + '/valid/dogs/'

# 构建训练数据集
train_cat_filenames = tf.constant([train_cats_dir + filename for filename in os.listdir(train_cats_dir)][:10000])
train_dog_filenames = tf.constant([train_dogs_dir + filename for filename in os.listdir(train_dogs_dir)][:10000])
train_filenames = tf.concat([train_cat_filenames, train_dog_filenames], axis=-1)
train_labels = tf.concat([
    tf.zeros(train_cat_filenames.shape, dtype=tf.int32),
    tf.ones(train_dog_filenames.shape, dtype=tf.int32)],
    axis=-1)

In [69]:
train_dog_filenames  # 文件路径

<tf.Tensor: shape=(10000,), dtype=string, numpy=
array([b'../../../../Other/datasets/cats_vs_dogs/train/dogs/dog.0.jpg',
       b'../../../../Other/datasets/cats_vs_dogs/train/dogs/dog.1.jpg',
       b'../../../../Other/datasets/cats_vs_dogs/train/dogs/dog.10.jpg',
       ...,
       b'../../../../Other/datasets/cats_vs_dogs/train/dogs/dog.8537.jpg',
       b'../../../../Other/datasets/cats_vs_dogs/train/dogs/dog.8538.jpg',
       b'../../../../Other/datasets/cats_vs_dogs/train/dogs/dog.8539.jpg'],
      dtype=object)>

In [70]:
train_labels

<tf.Tensor: shape=(20000,), dtype=int32, numpy=array([0, 0, 0, ..., 1, 1, 1])>

In [71]:
def _decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)  # 读取原始文件
    image_decoded = tf.image.decode_jpeg(image_string)  # 解码JPEG图片
    image_resized = tf.image.resize(image_decoded, [256, 256]) / 255.0
    return image_resized, label


train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))

In [72]:
%%timeit
train_dataset.map(map_func=_decode_and_resize,
                  num_parallel_calls=tf.data.AUTOTUNE)

13.2 ms ± 3.28 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Creates a Dataset that prefetches elements from this dataset.

Most dataset input pipelines should end with a call to prefetch.
This allows later elements to be prepared while the current element is being processed.
This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.

In [73]:
%%timeit
train_dataset.map(map_func=_decode_and_resize,
                  num_parallel_calls=tf.data.experimental.AUTOTUNE).\
    prefetch(tf.data.experimental.AUTOTUNE)  # 性能得到提升

10.6 ms ± 487 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
